In [1]:
import numpy as np
import pandas as pd
import math
import os
import pickle
import importlib as il


funcModule = "data_structure"
ds = il.import_module(funcModule)

stats = ds.stats
stats.countingStats = ['MIN','FG','FGA','s3P','s3PA','FT','FTA','OR','DR','TOT','A','PF','ST','TO','BL','PTS']

playerDB = {}

stats.countingStats

['MIN',
 'FG',
 'FGA',
 's3P',
 's3PA',
 'FT',
 'FTA',
 'OR',
 'DR',
 'TOT',
 'A',
 'PF',
 'ST',
 'TO',
 'BL',
 'PTS']

In [2]:
if os.path.exists("gameLogs.csv"):
    rawData = pd.read_csv("gameLogs.csv")
else:
    print("no such file found")
    
if os.path.exists("team_dict.csv"):
    teamDB = pd.read_csv("team_dict.csv")
else:
    print("no such file found")

teamDB.set_index("City Name", drop=True, inplace=True)

team_dict = teamDB['NBA.com Abbreviation'].to_dict()

gameDB = {}

In [3]:
colNames = list(rawData.columns.values)

i = 0

for i in range(len(colNames)):
    if colNames[i][0].isdigit():
        colNames[i] = "s" + colNames[i]
    colNames[i] = colNames[i].replace(" ","_")
   
colNames[6] = 'VENUE'
    
rawData.columns = colNames


In [19]:

own_vals = [team_dict[x] for x in rawData["OWN_TEAM"]]
opp_vals = [team_dict[x] for x in rawData["OPP_TEAM"]]
home = [(x=='H') for x in rawData["VENUE"]]
date = [x for x in rawData["DATE"]]

home_tm = []
away_tm = []
game_code = []

for i in range(len(own_vals)):
    if home[i]:
        home_tm.append(own_vals[i])
        away_tm.append(opp_vals[i])
    else:
        home_tm.append(opp_vals[i])
        away_tm.append(own_vals[i])
    game_code.append(date[i] + "#" + home_tm[i] + "#" + away_tm[i])

rawData = rawData.assign(own_team = own_vals)
rawData = rawData.assign(home_team = home_tm)
rawData = rawData.assign(away_team = away_tm)
rawData = rawData.assign(game_id = game_code)

rawData.iloc[:2,6:12]

,VENUE,MIN,FG,FGA,s3P,s3PA
0,R,30.1,8,18,1,4
1,R,32.8,5,13,3,5


In [20]:
playerCounter = 1

for eachName in rawData['PLAYER_FULL_NAME'].unique():
    playerDB[eachName] = ds.player({'name':eachName,'playerID':playerCounter})
    playerCounter += 1

playerDB[rawData["PLAYER_FULL_NAME"][0]].name

'Carmelo Anthony'

In [22]:
checkPt = -1
i = 0
entries = len(rawData)

for eachEntry in rawData.itertuples():
    thisPlayer = playerDB[eachEntry.PLAYER_FULL_NAME]
    thisGame = eachEntry.game_id
    
    if thisPlayer.gameStats is None:
        thisPlayer.gameStats = {}
    thisPlayer.gameStats[thisGame] = [eachEntry[eachEntry._fields.index(cStat)] for cStat in stats.countingStats]
    
    if thisGame in gameDB:
        gameDB[thisGame].gamePlayers.append(thisPlayer)
        gameDB[thisGame].gameStats[thisPlayer] = thisPlayer.gameStats
    else:
        gameDB[thisGame] = ds.game({'game_id': thisGame})
        gameDB[thisGame].gamePlayers.append(thisPlayer)
        gameDB[thisGame].gameStats[thisPlayer] = thisPlayer.gameStats
    
    if i == checkPt:
        break
    else:
        i += 1
        if i % 100 == 0:
            print("     processed game " + str(i) + " of " + str(entries))

     processed game 100 of 27874
     processed game 200 of 27874
     processed game 300 of 27874
     processed game 400 of 27874
     processed game 500 of 27874
     processed game 600 of 27874
     processed game 700 of 27874
     processed game 800 of 27874
     processed game 900 of 27874
     processed game 1000 of 27874
     processed game 1100 of 27874
     processed game 1200 of 27874
     processed game 1300 of 27874
     processed game 1400 of 27874
     processed game 1500 of 27874
     processed game 1600 of 27874
     processed game 1700 of 27874
     processed game 1800 of 27874
     processed game 1900 of 27874
     processed game 2000 of 27874
     processed game 2100 of 27874
     processed game 2200 of 27874
     processed game 2300 of 27874
     processed game 2400 of 27874
     processed game 2500 of 27874
     processed game 2600 of 27874
     processed game 2700 of 27874
     processed game 2800 of 27874
     processed game 2900 of 27874
     processed game 300